---

YAML

---

Esse *script* tem como objetivo obter os dados sobre mananciais gerenciados pela [SABESP](http://site.sabesp.com.br), que abastecem a Região Metropolitana de São Paulo (RMSP). A gestão desses dados é feita pelo Laboratório de Sistemas de Suporte a Decisões em Engenharia Ambiental e de Recursos Hídricos ([LabSid](http://www.labsid.eng.br)), que dá suporte ao Sistema de Suporte a Decisões [SSD](http://ssd3sabesp.labsid.eng.br), bem como publiciza os dados no [*site* da SABESP](http://mananciais.sabesp.com.br/HistoricoSistemas).

<br>

O *script* foi desenvolvido a partir da interpretação do arquivo *json* que alimenta o *site*, identificado inspecionando a página. Funções específicas foram criadas para obter cada conjunto de dados (conforme o arquivo *json* se encontra separado). Ao final, para obtenção dos resultados, as funções são aplicadas em conjunto, otendo-se os dados para um intervalo entre datas, ou ainda, uma data única, com a finalidade de uma atualização diária.
![json](https://i.imgur.com/nvlAMuz.png)

<br>

***ToDo***: Criar *script* para consumir esses dados:
- http://mananciais.sabesp.com.br/api/Mananciais/ResumoTelemetricos/2020-04-01

***ToDo***: O Presente código tem a finalidade de obter os dados unicamente do Sistema Cantareira e, portanto, não se pensou em adicionar a possibilidade de obter os dados dos outros sistemas produtores, por meio da inclusão e ajuste desse parâmetro definido e fixado como *0*.

# Imports

Inicialmente importa-se as bibliotecas que serão necessárias para rodar os códigos abaixo.

In [ ]:
import os
import json
import urllib.request
import pandas as pd
import datetime
import calendar
import time

E é criada a estrutura de pastas que será utilizada para armazenar os arquivos que serão criados ao longo do processo. Em específico, a pasta *data* é a que armazenará a tabela com as informações.

In [ ]:
%run '../codes/files/create_folders.py'

create_folders('', ['data', 'docs'])

# Série Histórica e Datas

A função abaixo retorna três parâmetros:
1. Tabela com uma série histórica, com uma data de início e uma data de término.
2. Data inicial em formato texto, para ser usada no nome do arquivo que vamos criar.
3. Data final em formato texto, para ser usada no nome do arquivo que vamos criar.

Caso não seja definida uma data final, será usado a data de hoje.
Importante ter atenção a isso pois a SABESP disponibiliza atualizações dos dados as 9:00 (conforme site deles) e, caso o presente código seja rodado entre a meia noite e as 9:00, tentar-se-á obter informações não disponíveis, provavelmente gerando erros. Para contornar isso, é necessário inserir, pelo menos, um parâmetro como *dia de hoje - 1*, por exemplo.

In [ ]:
def create_df(start='1970-1-1', end=None):
    """
    Function to create date table, with only on colum named 'Data' as index.
    With no 'end' parameter is passed, the function will return a table until today
    With no 'start' parameter is passed, the function will return a table staring in firts day of de 70's.
    
    The function return two more parameters to used in filenames:
    filename_start > first day of table 
    filename_end   > last day of table as string
    
    """
    
    # Dataframe to get dates
    if end is None:
        df = pd.DataFrame(pd.date_range(pd.to_datetime(start), end=datetime.date.today()), columns=['Data'])
        filename_start = str(datetime.datetime.strptime(str(start), '%Y-%m-%d').strftime('%Y.%m.%d'))
        filename_end   = str(datetime.datetime.strptime(str(datetime.date.today()), '%Y-%m-%d').strftime('%Y.%m.%d'))
    
    else:
        df = pd.DataFrame(pd.date_range(pd.to_datetime(start), end=pd.to_datetime(end)), columns=['Data'])
        filename_start = str(datetime.datetime.strptime(str(start), '%Y-%m-%d').strftime('%Y.%m.%d'))
        filename_end   = str(datetime.datetime.strptime(str(end), '%Y-%m-%d').strftime('%Y.%m.%d'))

    # Results
    df = df.set_index('Data')
    return df, filename_start, filename_end

In [ ]:
start = '2020-02-01'
end = '2020-02-10'

df_day, filename_start, filename_end = create_df(start, end)

In [ ]:
print(filename_start)
print(filename_end)

# *Link* para fazer o *download* do Json

Inicialmente foram feitas tentativas diversas para melhor conhecimento do [*site* da SABESP](http://mananciais.sabesp.com.br/HistoricoSistemas?SistemaId=0) que disponibiliza as informações dos mananciais. Inicialmente tentou-se obter os dados pela técnica de *web scrapping*, até que descobriu-se que os dados são distribuídos por meio de uma API do SSD (Sistema de Suporte a Decisões).

Observou-se que a consulta manual apresenta os dados do mês da presente data até o primeiro dia do mês anterior. Por exemplo, se estamos no dia 25.**03**.2020, os dados apresentados serão dessa data até o dia 01.**02**.2020, retornando aproximadamente dados de 55 dias (30 dias de um mês hipotético e 25 do outro). O mesmo padrão irá ocorrer caso a consulta seja feita em 01.**03**.2020, a qual retornar-a os dados *até o primeiro dia do mês anterior*, ou seja, 01.**02**.2020.

Essa forma de "entregar" os dados foi considerada na requisição de dados pela API, a qual foi feita com uso do [urllib.request](https://stackoverflow.com/questions/32795460/loading-json-object-in-python-using-urllib-request-and-json-modules). A API tem seu *link* padrão apresentado abaixo, sendo inserido apenas duas variáveis: a data e o Sistema (aqui representado pelo 0 no final, que representa o Sistema Produtor Cantareira).

- http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/2020-03-25/0

In [ ]:
site = ('http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/' + 
        str(filename_start.replace('.','-')) + '/' +
        str(filename_end.replace('.','-')) + '/0')

site

# Convertendo Json para tabela e extraíndo dados

Com o *link* da API com uma data definida, criou-se uma função para obter os dados em formato *json*, bastando inserir o site.

In [ ]:
def get_json(url):
    # Get Array with data
    webURL = urllib.request.urlopen(url)
    my_bytes = webURL.read()

    # Transform Array into JSON
    my_json = my_bytes.decode('utf8')
    data = json.loads(my_json)
    return json.dumps(data, indent=2, sort_keys=True)

In [ ]:
jsn = get_json(site)
#print(jsn)

Com o arquivo *json* contendo todos os dados do periodo selecionado, com diversas chaves e subchaves, iniciou-se a segmentação do arquivo, convertendo o arquivo para uma tabela, com a qual tenho mais familiaridade para editar e filtrar.

A função abaixo tem esse objetivo e já aproveita para excluir duas colunas que, aparentemente, não agregam informações.

In [ ]:
def json2df(jsn):
    # Create dataframe
    df = pd.read_json(jsn)

    # Delete columns
    return df.drop(['FlagHasError', 'Message'], axis=1)

In [ ]:
df = json2df(jsn)
df

## Sistema Produtor (*json: SistemaId*)

Por meio do campo *SistemaId* é possível obter o código que define qual é o sistema produtor de água. Contudo, considerando que o presente *script* visa obter somente os dados do Sistema Cantareira, **tal função não será aplicada**.

In [ ]:
def get_system(df):
    # JSON to dataframe    
    data = df.loc['SistemaId']['ReturnObj']
    
    # Results
    return data

In [ ]:
get_system(df)

## Data Final  (*json: DataFinal*)

Identifica a data final empregada na API. Não se vislumbra muita utilidade para essa informação nesse momento, tendo em vista que foi o usuário que definiu esse parâmetro na definição do *link* de acesso à API. Logo, **tal função não será aplicada no código final**.

In [ ]:
def get_enddate(df):
    # JSON to dataframe    
    data = df.loc['DataFinal']['ReturnObj']   
    
    # Results
    return pd.to_datetime(data, dayfirst=True).strftime('%Y-%m-%d')

In [ ]:
data = get_enddate(df)
data

## Data Inicial  (*json: DataInicial*)

Identifica a data inicial empregada na API. Não se vislumbra muita utilidade para essa informação nesse momento, tendo em vista que foi o usuário que definiu esse parâmetro na definição do *link* de acesso à API. Logo, **tal função não será aplicada no código final**.

In [ ]:
def get_startdate(df):
    # JSON to dataframe    
    data = df.loc['DataInicial']['ReturnObj']   
    
    # Results
    return pd.to_datetime(data, dayfirst=True).strftime('%Y-%m-%d')

In [ ]:
data = get_startdate(df)
data

## Manobras Operacionais (*json: ListaManobras*)

Identifica todas as manobras listadas no site da SABESP. São dados mais descritivos, disponibilizados visando dar mais transparência a cadeia de comando para abrir ou fechar os reservatórios. Nesse primeiro momento tais dados não serão analisado e, portanto, **tal função não será aplicada**.

In [ ]:
def get_manobras(df):
    # JSON to dataframe    
    lst = df.loc['ListaManobras']['ReturnObj']
    
    # Results
    return pd.json_normalize(lst)

In [ ]:
df_manobras = get_manobras(df)
df_manobras

## Componentes do Sistema (*json: ListaRepresas*)

### Reservatórios

A função abaixo lista os reservatórios (ou represas) do Sistema Cantareira e outras que estão integradas, de alguma maneira, ao Sistema, inserindo também o identificador de cada reservatório (*ComponenteId*).

Apesar de trata-se de uma tabela que não retorna dados temporais (por exemplo: vazão, volume e chuva), ou seja, que variam ao longo do tempo, é fundamental para rotular de qual reservatório que são os dados temporais que serão obtidos nas próximas funções, visto que eles se valem, majoritariamente, do campo *ComponenteId*.

In [ ]:
def list_represas(df):
    # JSON to dataframe
    lst = df.loc['ListaRepresas']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['temChuva','temNivel', 'temQjus', 'temQnat', 'temVolume'], axis=1)

    # Results
    return df

In [ ]:
df_represas = list_represas(df)
df_represas

### Estruturas (Túneis e outros Pontos de Medição)  (*json: ListaLocais*)

A função abaixo lista os túneis e estações de monitoramento do Sistema Cantareira e outras que estão integradas, de alguma maneira, ao Sistema, inserindo também o identificador de cada local (*ComponenteId*).

Apesar de trata-se de uma tabela que não retorna dados temporais (por exemplo: vazão, volume e chuva), ou seja, que variam ao longo do tempo, contudo é fundamental para rotular de qual estrutura que são os dados temporais que serão obtidos nas próximas funções, visto que eles se valem, majoritariamente, do campo *ComponenteId* ou *abreviatura*.

In [ ]:
def list_estruturas(df):
    # JSON to dataframe
    lst = df.loc['ListaLocais']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['Maximo','Minimo',
                  'Data','Dia',
                  'Valor','Unidade'],
                 axis=1)


    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('ComponenteId')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Results
    return df

In [ ]:
df_estruturas = list_estruturas(df)
df_estruturas

## Dados Diários

Nessa seção que serão obtidos diversos dados relevantes na operação do Sistema Cantareira, tais como:
- Vazão natural em cada reservatório;
- Vazão afluente em cada reservatório;
- Vazão defluente em cada reservatório;
- Nível e Volume de cada reservatório;
- Dados de Precipitação de cada reservatório.

Inicialmente, definiu-se uma função para renomear *strings*, visto que estas constarão nos cabeçalhos das tabelas a serem criadas. Aplicou-se a função na *df_represas* (criada acima) apenas para observar quais serão os nomes que constarão no cabeçalho das tabelas.

In [ ]:
def rename_field(x):
    return(x.replace('/', '-').
           replace(' (', '-').
           replace('-', '_').
           replace(')', '').
           replace('Cesp', 'CESP').
           replace('Represa ', '').
           replace(' ', '')
          )

### Volume, QJusante e Chuva (*json: ListaDados*)

Extraíndo os dados do json, por meio da chave *ListaDados* e subchave *Dados*, foi obtido os dados de volume, vazão defluente e precpitação de cada reservatório.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome do reservatório. Portanto, foi necessário filtrar essas tabelas por reservatório, ajustar o cabeçalho inserindo o nome do reservatório em questão e, posteriromente, fazer um join das tabelas pelo campo *data*.

In [ ]:
def list_volumes(df):
    # JSON to dataframe
    lst = df.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')

    # Define pivot to create new tables
    fields = df['Nome']
    fields = sorted(list(set(fields)))

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = create_df()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        df_name = 'df_dados' + '_' + j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[df_name] = df[df['Nome'] == i]

        # Deleta colunas
        locals()[df_name] = locals()[df_name].drop(['FlagConsolidado',
                                                    'NAMaxMax','NAMinMin',
                                                    'QJusanteMax','QJusanteMin',
                                                    'NivelUltimoDia',
                                                    'SistemaId','ComponenteId',
                                                    'UltimoDia',
                                                    'VazaoJusantePrincipal','VazaoJusanteSecundaria',
                                                    'VolumeOperacionalUltimoDia','VolumePorcentagemUltimoDia',
                                                    'VolumeTotalUltimoDia','Nome'], axis=1)

        # Renomeia as colunas
        locals()[df_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[df_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[df_name]['Data'] = pd.to_datetime(locals()[df_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full, locals()[df_name], on='Data', how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    return df_full

In [ ]:
df_volumes = list_volumes(df)
display(df_volumes)

### Vazão Afluente e Vazão Natural  (*json: ListaDados*)

Extraíndo os dados do json, por meio da chave *ListaDados* e subchave *Qnat*, foram obtido os dados de vazão afluente e vazão naturalde cada reservatório.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome do reservatório. Portanto, foi necessário filtrar essas tabelas por reservatório, ajustar o cabeçalho inserindo o nome do reservatório em questão e, posteriromente, fazer um join das tabelas pelo campo *data*.

In [ ]:
def list_vazao(df):
    # Represas
    df_represas = list_represas(df)
    
    # JSON to dataframe
    lst = df.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst, 'Qnat')

    # Merge Tables
    df = pd.merge(df, df_represas, on='ComponenteId', how='outer')

    # Define pivot to create new tables
    fields = df['Nome']
    fields = sorted(list(set(fields)))
    
    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = create_df()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        df_name = 'df_vazao'+'_'+j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[df_name] = df[df['Nome'] == i]

        # Deleta colunas
        locals()[df_name] = locals()[df_name].drop(['ComponenteId','Nome',
                                                    'VazaoAfluenteMax','VazaoAfluenteMin',
                                                    'VazaoNaturalMax','VazaoNaturalMin'], axis=1)

        # Renomeia as colunas
        locals()[df_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[df_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[df_name]['Data'] = pd.to_datetime(locals()[df_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[df_name], on='Data', how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    return df_full

In [ ]:
df_vazao = list_vazao(df)
display(df_vazao)

### Sistema Equivalente  (*json: ListaDados*)

Extraíndo os dados do *json*, por meio da chave *ListaDados*, foram obtido os dados do Sistema Equivalente.
A SABESP entendo que o Sistema Equivalente é a somatária das represas que estão na Bacia do rio Piracicaba, ou seja:
- Jaguari/Jacareí;
- Cachoeira;
- Atibainha.

In [ ]:
def list_SE(df):
    # JSON to dataframe
    lst = df.loc['ListaDados']['ReturnObj']
    df = pd.json_normalize(lst)

    # Delete columns
    df = df.drop(['Dados','Data','Qnat'], axis=1)

    # Transform columns to list and reorder list
    col = df.columns.to_list()

    # Functions to rename
    col = ['SE_'+x for x in col]
    col = [x.replace('SistemaEquivalente.', '').replace('SE_Data', 'Data') for x in col]

    # Rename Columns
    df.columns = col

    # Convert Data Column (object) to datatime colum
    df['Data'] = pd.to_datetime(df['Data'])

    # Results
    df = df.set_index('Data')
    df.dropna(how='all', inplace=True)
    
    return df

In [ ]:
df_SE = list_SE(df)
display(df_SE)

### Sistema Cantareira (*json: ListaDadosSistema*)

In [ ]:
def list_SC(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosSistema']['ReturnObj']
    df = pd.json_normalize(lst)
    
    # Delete columns
    df = df.drop(['SistemaId'], axis=1)
    
    # Transform columns to list and reorder list
    col = df.columns.to_list()

    # Functions to rename
    col = ['SC_'+x for x in col]
    col = [x.replace('SC_Data', 'Data') for x in col]

    # Rename Columns
    df.columns = col
    
    # Convert Data Column (object) to datatime colum
    df['Data'] = pd.to_datetime(df['Data'])

    # Results
    df = df.set_index('Data')
    df.dropna(how='all', inplace=True)

    # Results
    return df

In [ ]:
df_SC = list_SC(df)
df_SC

### Vazão dos Túneis e outros Pontos de Medição (*json: ListaDadosLocais*)

Extraíndo os dados do json, por meio da chave *ListaDadosLocais* e subchave *Dados*, foram obtido os dados de vazão dos túneis Q7, Q6 Q5 e outros.

No arquivo json, as tabelas encontravam-se empilhadas (*flat table*), com uma coluna com o nome da estrutura. Portanto, foi necessário filtrar essas tabelas por estrutura, adicionando ao cabeçalho seu repectivo nome e, posteriormente, fazer um join das tabelas pelo campo *data*.

In [ ]:
def list_vazaoestruturas(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosLocais']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')

    # Define pivot to create new tables
    fields = df['Abreviatura']
    fields = sorted(list(set(fields)))

    # Transform columns to list and reorder list
    col = df.columns.to_list()
    col.insert(0, col.pop(col.index('Data')))
    col.append(col.pop(col.index('Unidade')))

    # Reindex Columns
    df = df.reindex(columns=col)

    # Create a blank table
    df_full,start,end = create_df()

    for i in fields:
        # Define Nomes e Nomes de Tabelas
        j = rename_field(i)
        df_name = 'df_dados' + '_' + j
        
        # Filtra e cria das tabelas por fields (represas)
        locals()[df_name] = df[df['Abreviatura'] == i]

        # Deleta colunas
        locals()[df_name] = locals()[df_name].drop(['Maximo', 'Minimo', 'Dia',
                                                     'Abreviatura', 'ComponenteId',
                                                    'LocalMedicaoId', 'Nome', 
                                                    'SistemaId'], axis=1)

        # Renomeia as colunas
        locals()[df_name].columns = [x if x=='Data' else j+'_'+x for x in locals()[df_name].columns]

        # Convert Data Column (object) to datatime colum
        locals()[df_name]['Data'] = pd.to_datetime(locals()[df_name]['Data'])

        # Merge all tables
        df_full = pd.merge(df_full,locals()[df_name], on='Data', how='left')

    # Results
    df_full = df_full.set_index('Data')
    df_full.dropna(how='all', inplace=True)
    
    return df_full

In [ ]:
df_vazaoestruturas = list_vazaoestruturas(df)
display(df_vazaoestruturas)

## Dados Horários

Referem-se a transposição da bacia do rio Paraíba do Sul para a bacia do rio Piracicaba, por meio da Estação Elevatória de Água Bruta (EEAB) Jaguari, que despeja água na represa Atibainha.

Tais dados não serão aqui considerados, visto que já se encontram discretizados em dado diário na tabela acima. Logo, **tal função não será aplicada**.

### Estrutura de Transposição (*json: ListaEspecial*)

In [ ]:
def list_EEAB(df):
    # JSON to dataframe
    lst = df.loc['ListaEspecial']['ReturnObj']
    df = pd.json_normalize(lst)
    
    # Results
    return df

In [ ]:
df_vazaoEEAB_pontos = list_EEAB(df)
df_vazaoEEAB_pontos

### Vazão de Transposição  (*json: ListaDadosEspecial*)

Dados horários, transferência 

In [ ]:
def list_vazao_EEAB(df):
    # JSON to dataframe
    lst = df.loc['ListaDadosEspecial']['ReturnObj']
    df = pd.json_normalize(lst, 'Dados')
    
    # Results
    return df

In [ ]:
df_vazaoEEAB = list_vazao_EEAB(df)
df_vazaoEEAB

# Resultado: Série Histórica do Sistema Cantareira

Com todas as funções definidas, é possível aplicar tais funções sequencialmente, visando criar uma série histórica com todos os dados do Sistema Cantareira.

## Período

In [ ]:
# Define data de início e fim
start = '2020-1-1'
end = '2020-5-18'

# Roda a primeira função para pegar a tabela com datas
df_day, filename_start, filename_end = create_df(start, end)

# Years's List
list_year = df_day.index.year
list_year = list(set(list_year))
list_year = sorted(list_year, reverse = True)
print(list_year)

# Zera os Objetos
dfs_volumes         = []
dfs_vazao           = []
dfs_SE              = []
dfs_SC              = []
dfs_vazaoestruturas = []

# Function to loop
for y in list_year:
    # Tempo
    print('Início do ano ' + str(y) + ' as ' + datetime.datetime.now().strftime('%H:%M:%S'))
    
    # Variáveis de Data
    firstdayyear   = datetime.date(y, 1, 1)
    lastdayyear    = datetime.date(y, 12, 31)
    today          = datetime.date.today()
    
    if today < lastdayyear:
        lastday = today
    elif today >= lastdayyear:
        lastday = lastdayyear
        
    # Site
    site = ('http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/' + 
            str(firstdayyear).replace('.','-') + '/' + 
            str(lastday).replace('.','-') + '/0')
        
    # Get json
    jsn                 = get_json(site)
    time.sleep(3)
    df                  = json2df(jsn)
    
    # Data    
    system              = get_system(df)
    startdate           = get_startdate(df)
    enddate             = get_enddate(df)    
    df_manobras         = get_manobras(df)
    
    # Represas
    df_represas         = list_represas(df)
    
    # Dados
    df_volumes          = list_volumes(df)
    df_vazao            = list_vazao(df)
    df_SE               = list_SE(df)
    df_SC               = list_SC(df)
    df_vazaoestruturas  = list_vazaoestruturas(df)
    
    # Concat Data
    dfs_volumes.append(df_volumes)
    dfs_vazao.append(df_vazao)
    dfs_SE.append(df_SE)
    dfs_SC.append(df_SC)
    dfs_vazaoestruturas.append(df_vazaoestruturas)
    
# Time
print('Fim as ' + datetime.datetime.now().strftime('%H:%M:%S'))

### Conctatena e une tabelas

In [ ]:
# Concat Data
dfs_volumes         = pd.concat(dfs_volumes)
dfs_vazao           = pd.concat(dfs_vazao)
dfs_SE              = pd.concat(dfs_SE)
dfs_SC              = pd.concat(dfs_SC)
dfs_vazaoestruturas = pd.concat(dfs_vazaoestruturas)

# One Table
dfs_volumes = pd.concat([dfs_volumes, dfs_vazao], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_SE], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_SC], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_vazaoestruturas], axis=1)

# Merge
df_final = pd.merge(df_day, dfs_volumes, left_index=True, right_index=True, how='left')
display(df_final)

### Export to CSV

In [ ]:
# Export
df_final.dropna(how='all', inplace=True)

filename = 'tab_Cantareira_' + filename_start + '_até_' + filename_end + '.csv'
df_final.to_csv(os.path.join('data', filename),
                index=True,
                header=True,
                encoding='UTF-8-SIG',
                sep=';',
                decimal=',',
                date_format='%d/%m/%Y')

## Apenas o dia de hoje

In [ ]:
# Define data de início e fim
start = datetime.date.today()
end = datetime.date.today()

# Roda a primeira função para pegar a tabela com datas
df_day, filename_start, filename_end = create_df(start, end)

# Zera os Objetos
dfs_volumes         = []
dfs_vazao           = []
dfs_SE              = []
dfs_SC              = []
dfs_vazaoestruturas = []

# Tempo
print('Início do ano as ' + datetime.datetime.now().strftime('%H:%M:%S'))
    
# Variáveis de Data
today          = datetime.date.today()
    
# Site
site = ('http://mananciais.sabesp.com.br/api/Mananciais/RepresasSistemasNivel/' + 
        str(start).replace('.','-') + '/' + 
        str(end).replace('.','-') + '/0')
        
# Get json
jsn                 = get_json(site)
time.sleep(3)
df                  = json2df(jsn)

# Data    
system              = get_system(df)
startdate           = get_startdate(df)
enddate             = get_enddate(df)    
df_manobras         = get_manobras(df)

# Represas
df_represas         = list_represas(df)
    
# Dados
df_volumes          = list_volumes(df)
df_vazao            = list_vazao(df)
df_SE               = list_SE(df)
df_SC               = list_SC(df)
df_vazaoestruturas  = list_vazaoestruturas(df)

# Concat Data
dfs_volumes.append(df_volumes)
dfs_vazao.append(df_vazao)
dfs_SE.append(df_SE)
dfs_SC.append(df_SC)
dfs_vazaoestruturas.append(df_vazaoestruturas)
   
# Time
print('Fim as ' + datetime.datetime.now().strftime('%H:%M:%S'))

### Conctatena e une tabelas

In [ ]:
# Concat Data
dfs_volumes         = pd.concat(dfs_volumes)
dfs_vazao           = pd.concat(dfs_vazao)
dfs_SE              = pd.concat(dfs_SE)
dfs_SC              = pd.concat(dfs_SC)
dfs_vazaoestruturas = pd.concat(dfs_vazaoestruturas)

# One Table
dfs_volumes = pd.concat([dfs_volumes, dfs_vazao], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_SE], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_SC], axis=1)
dfs_volumes = pd.concat([dfs_volumes, dfs_vazaoestruturas], axis=1)

# Merge
df_final = pd.merge(df_day, dfs_volumes, left_index=True, right_index=True, how='left')
display(df_final)

### Export to CSV

In [ ]:
# Export
df_final.dropna(how='all', inplace=True)

filename = 'tab_Cantareira_' + filename_start + '_até_' + filename_end + '.csv'
df_final.to_csv(os.path.join('data', filename),
                index=True,
                header=True,
                encoding='UTF-8-SIG',
                sep=';',
                decimal=',',
                date_format='%d/%m/%Y')